In [40]:
# pre-install and import all the libraries in one go
import numpy as np
import ee   
import numpy as np
import os
import time
import pandas as pd
import geemap
import geemap.colormaps as cm

In [41]:
try:
  ee.Initialize()
except:
  ee.Authenticate()
  ee.Initialize()

In [42]:
ee.data.setDefaultWorkloadTag('ee-gee-test-notebook2-evandelancey')

# Stacking data and aggregating

In [43]:
#california
california = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM1_NAME', 'California'))

#data for stacking
elevation = ee.Image("NASA/NASADEM_HGT/001").select('elevation') # elevation 30 m
slope = ee.Terrain.slope(elevation).rename('slope')
climate = ee.ImageCollection("ECMWF/ERA5/MONTHLY").filterDate('2020-04-01', '2020-05-01')
temp = climate.select('mean_2m_air_temperature').mean().subtract(273.15).rename('temp')
soilNit = ee.Image("projects/soilgrids-isric/nitrogen_mean").select('nitrogen_0-5cm_mean').divide(100).rename('soilNit')

Map = geemap.Map()
Map.centerObject(california, 6)
Map.addLayer(temp.clip(california), {'min': 5, 'max':25, 'palette': cm.get_palette('jet', n_class=8)}, 'temp')
Map.addLayer(elevation.clip(california), {'min':0, 'max':3000, 'palette': cm.palettes.dem}, 'elevtation')
Map

Map(center=[37.182082045953905, -119.46872761017087], controls=(WidgetControl(options=['position', 'transparen…

In [44]:
#stack data
dataStack = elevation.addBands(slope).addBands(temp).addBands(soilNit)
print(dataStack.bandNames().getInfo())

['elevation', 'slope', 'temp', 'soilNit']


In [61]:
#summarize data by hexagons on the fly
i = ee.Image(1).clip(california)
seeds = ee.Algorithms.Image.Segmentation.seedGrid(30, 'hex')
grid = ee.Algorithms.Image.Segmentation.SNIC(image = i, seeds = seeds)
grid = grid.select('clusters')
hexMeans =  dataStack.addBands(grid).reduceConnectedComponents(ee.Reducer.mean(), 'clusters', 256)



Map = geemap.Map()
Map.centerObject(california, 6)
Map.addLayer(grid.randomVisualizer(), None, 'Hex grid')
Map.addLayer(hexMeans, {'min':0, 'max': 20, 'bands': ['slope'], 'palette': cm.get_palette('magma_r', n_class=8)}, 'hex mean slope')
Map.addLayer(hexMeans, {'min':0, 'max': 40, 'bands': ['soilNit'], 'palette': cm.get_palette('pink_r', n_class=8)}, 'hex mean soil nitrogen')
Map

Map(center=[37.182082045953905, -119.46872761017087], controls=(WidgetControl(options=['position', 'transparen…

# Exporting data

In [67]:
#export gridded data
task = ee.batch.Export.image.toAsset(image=hexMeans,
                                     description='environmentVars_hexGrid',
                                     assetId = 'projects/bcs-loc360-earth-engine-dev/assets/TestDump/environmentVars_hexGrid',
                                     scale=30,
                                     region=california.geometry(),
                                    maxPixels=10e12)
task.start()